In [1]:
import pandas as pd
import numpy as np

from os.path import join
from eyefeatures.features.stats import SaccadeFeatures, FixationFeatures, RegressionFeatures, MicroSaccades
from eyefeatures.features.measures import HurstExponent, ShannonEntropy
from eyefeatures.features.extractor import Extractor

DATA_PATH = join('..', 'data', 'fixations')

In [2]:
data = pd.concat([pd.read_excel(join(DATA_PATH, 'itog_fix_1.xlsx')),
                  pd.read_excel(join(DATA_PATH, 'itog_fix_2.xlsx'))],
                 axis=0)

In [3]:
data['AOI'] = 'aoi_1'

for i in data.index[::5]:
    data.loc[i, 'AOI'] = 'aoi_2'
for i in data.index[::6]:
    data.loc[i, 'AOI'] = 'aoi_2'
for i in data.index[::7]:
    data.loc[i, 'AOI'] = 'aoi_2'

In [4]:
data['AOI'].value_counts()

AOI
aoi_2    5304
aoi_1    3233
Name: count, dtype: int64

In [5]:
x = 'norm_pos_x'
y = 'norm_pos_y'
t = 'start_timestamp'
dur = 'duration'
dis = 'dispersion'


sac_feats_stats = {
    'length': ['min', 'max'],
    'speed': ['mean', 'kurtosis'],
    'acceleration': ['mean']
}

sf = SaccadeFeatures(x=x, y=y, t=t,
                     pk=['Participant', 'tekst'],
                     features_stats=sac_feats_stats)

fix_feats_stats = {'duration': ['sum'], 'vad': ['mean']}

fx = FixationFeatures(duration=dur, dispersion=dis,
                     pk=['Participant', 'tekst'],
                     aoi='AOI',
                     features_stats=fix_feats_stats)

rg = RegressionFeatures(x=x, y=y, t=t, duration=dur,
                        pk=['Participant', 'tekst'],
                        aoi='AOI',
                        features_stats=sac_feats_stats,
                        rule=(1, 3))

ms = MicroSaccades(x=x, y=y, t=t, dispersion=dis,
                   pk=['Participant', 'tekst'],
                   aoi='AOI',
                   features_stats=sac_feats_stats,
                   min_dispersion=0.01,
                   max_speed=0.2)

features = [
    SaccadeFeatures(features_stats=sac_feats_stats),
    FixationFeatures(features_stats=fix_feats_stats),
    RegressionFeatures(features_stats=sac_feats_stats, rule=(1, 3)),
    MicroSaccades(features_stats=sac_feats_stats, min_dispersion=0.001, max_speed=0.5)
]

extractor_ = Extractor(features=features, x=x, y=y, t=t, duration=dur, dispersion=dis,
                      aoi='AOI',
                      pk=['Participant', 'tekst'],
                      return_df=True)

In [6]:
sac_feats_stats = {
    'length': ['min', 'max'],
    'speed': ['mean', 'kurtosis'],
    'acceleration': ['mean']
}

fix_feats_stats = {'duration': ['sum'], 'vad': ['mean']}

features = [
    SaccadeFeatures(features_stats=sac_feats_stats),
    FixationFeatures(features_stats=fix_feats_stats),
    RegressionFeatures(features_stats=sac_feats_stats, rule=(1, 3)),
    HurstExponent(), ShannonEntropy()
]

extractor = Extractor(features=features, x=x, y=y, t=t, duration=dur, dispersion=dis,
                      aoi='AOI',
                      pk=['Participant', 'tekst'],
                      return_df=True)

In [7]:
sf.fit_transform(data)

,sac_length_min,sac_length_max,sac_speed_mean,sac_speed_kurtosis,sac_acceleration_mean
1_1,0.002657,0.433486,0.488956,5.498018,1.522857
1_2,0.003757,0.321837,0.451773,8.356764,1.507806
1_3,0.003663,0.365776,0.390753,7.639049,1.292694
1_4,0.000212,0.342315,0.333495,9.593384,0.999054
1_5,0.002705,0.375434,0.360223,9.717641,1.187418
...,...,...,...,...,...
2_33,0.001920,0.406550,0.264515,10.235914,0.945929
2_34,0.007928,0.354386,0.298396,9.732734,0.883503
2_35,0.002170,1.269198,0.623824,45.476927,2.527952
2_36,0.002196,1.124204,0.443600,10.414252,1.650884


In [8]:
fx.fit_transform(data)

,fix_duration_AOI[aoi_2]_sum,fix_vad_AOI[aoi_2]_mean,fix_duration_AOI[aoi_1]_sum,fix_vad_AOI[aoi_1]_mean
1_1,10760.106500,1.084216,2504.5290,1.173406
1_2,9308.563500,1.099930,1652.0625,1.427237
1_3,9218.071000,1.219697,1645.0175,1.014305
1_4,7324.355999,1.125739,952.0735,1.285798
1_5,9060.284000,1.148466,1942.1885,1.344456
...,...,...,...,...
2_33,13896.132000,1.376585,2428.7540,1.332026
2_34,5525.225500,1.291868,891.6540,1.522308
2_35,9431.293500,1.414826,1157.4960,1.392368
2_36,5885.597000,1.333476,614.9980,1.361147


In [9]:
rg.fit_transform(data)

,reg_length_AOI[aoi_2]_min,reg_length_AOI[aoi_2]_max,reg_speed_AOI[aoi_2]_mean,reg_speed_AOI[aoi_2]_kurtosis,reg_acceleration_AOI[aoi_2]_mean,reg_length_AOI[aoi_1]_min,reg_length_AOI[aoi_1]_max,reg_speed_AOI[aoi_1]_mean,reg_speed_AOI[aoi_1]_kurtosis,reg_acceleration_AOI[aoi_1]_mean
1_1,0.011841,0.420153,3.480621,4.196115,101.520275,0.037026,0.272665,3.396826,-1.593181,54.131503
1_2,0.006408,0.321837,3.276942,-0.922803,70.596140,0.030813,0.261686,2.167320,-0.934722,35.216219
1_3,0.005249,0.265067,2.629076,1.171818,73.916643,0.016539,0.254050,3.206941,3.553356,89.722992
1_4,0.009554,0.277172,3.406318,-0.759758,107.220463,0.041129,0.049056,2.288800,NaN,57.301368
1_5,0.002705,0.300800,2.548291,1.412943,76.290851,0.042946,0.104575,2.593522,0.004752,57.119943
...,...,...,...,...,...,...,...,...,...,...
2_33,0.007210,0.202824,2.316939,1.628924,97.018484,0.025199,0.221629,3.673611,-0.042142,148.173164
2_34,0.013271,0.303437,3.315178,0.574263,161.015426,0.023785,0.114924,3.094738,-1.646757,132.171743
2_35,0.007504,1.269198,5.238658,6.259332,186.293178,0.003259,0.559806,2.653471,-1.497979,54.424245
2_36,0.009415,0.504216,3.304351,9.638348,132.919882,0.017941,1.124204,6.391043,6.352378,142.120547


In [10]:
ms.fit_transform(data)

,microsac_length_AOI[aoi_2]_min,microsac_length_AOI[aoi_2]_max,microsac_speed_AOI[aoi_2]_mean,microsac_speed_AOI[aoi_2]_kurtosis,microsac_acceleration_AOI[aoi_2]_mean,microsac_length_AOI[aoi_1]_min,microsac_length_AOI[aoi_1]_max,microsac_speed_AOI[aoi_1]_mean,microsac_speed_AOI[aoi_1]_kurtosis,microsac_acceleration_AOI[aoi_1]_mean
1_1,0.011581,0.190765,0.345077,1.952940,1.167471,0.016593,0.198150,0.287073,-0.298782,0.660408
1_2,0.006408,0.176232,0.318633,1.040025,1.003533,0.014979,0.117818,0.310151,0.457796,1.040075
1_3,0.005065,0.185019,0.245316,14.807231,0.762113,0.003663,0.063193,0.194864,1.458462,0.744146
1_4,0.001980,0.110651,0.241609,7.668473,0.813152,0.041129,0.070823,0.313787,-1.606570,1.077196
1_5,0.002705,0.192296,0.301805,3.149362,0.942590,0.020778,0.104575,0.359371,-0.409101,1.326198
...,...,...,...,...,...,...,...,...,...,...
2_33,0.003962,0.181661,0.270166,6.836550,1.088828,0.016619,0.129722,0.220889,-1.668084,0.902597
2_34,0.011230,0.063878,0.186453,3.302904,0.750266,0.023785,0.114924,0.161013,1.929328,0.296183
2_35,0.007504,0.166901,0.327634,5.700887,1.533474,0.003259,0.180431,0.399833,-1.697831,1.281691
2_36,0.004319,0.154740,0.287022,8.486159,1.109342,0.017941,0.166901,0.355342,-0.564187,0.951186


In [11]:
extractor.fit_transform(data).head()

100%|██████████| 5/5 [00:00<00:00, 11.91it/s]


,sac_length_AOI[aoi_2]_min,sac_length_AOI[aoi_2]_max,sac_speed_AOI[aoi_2]_mean,sac_speed_AOI[aoi_2]_kurtosis,sac_acceleration_AOI[aoi_2]_mean,sac_length_AOI[aoi_1]_min,sac_length_AOI[aoi_1]_max,sac_speed_AOI[aoi_1]_mean,sac_speed_AOI[aoi_1]_kurtosis,sac_acceleration_AOI[aoi_1]_mean,...,reg_speed_AOI[aoi_2]_mean,reg_speed_AOI[aoi_2]_kurtosis,reg_acceleration_AOI[aoi_2]_mean,reg_length_AOI[aoi_1]_min,reg_length_AOI[aoi_1]_max,reg_speed_AOI[aoi_1]_mean,reg_speed_AOI[aoi_1]_kurtosis,reg_acceleration_AOI[aoi_1]_mean,hurst_exponent,entropy
1_1,0.011581,0.420153,3.316192,2.582125,88.408114,0.016593,0.272665,3.213086,-0.278472,68.252408,...,3.480621,4.196115,101.520275,0.037026,0.272665,3.396826,-1.593181,54.131503,0.014809,0.939931
1_2,0.006408,0.321837,3.007439,-0.464949,85.793710,0.014979,0.261686,2.103520,-0.316122,54.794634,...,3.276942,-0.922803,70.596140,0.030813,0.261686,2.167320,-0.934722,35.216219,0.010948,0.940286
1_3,0.005065,0.365776,2.959531,6.020307,94.441713,0.003663,0.254050,2.827255,0.763153,76.291863,...,2.629076,1.171818,73.916643,0.016539,0.254050,3.206941,3.553356,89.722992,0.020146,0.947435
1_4,0.001980,0.277172,3.005369,0.249804,121.601653,0.041129,0.272092,2.801384,2.951833,53.823491,...,3.406318,-0.759758,107.220463,0.041129,0.049056,2.288800,NaN,57.301368,0.017824,0.933634
1_5,0.002705,0.300800,2.426858,1.677840,69.126721,0.020778,0.104575,2.409873,0.137052,68.725060,...,2.548291,1.412943,76.290851,0.042946,0.104575,2.593522,0.004752,57.119943,0.009721,0.941102


In [12]:
rg_angles = RegressionFeatures(x=x, y=y, t=t, duration=dur,
                        pk=['Participant', 'tekst'],
                        aoi='AOI',
                        features_stats=sac_feats_stats,
                        rule=(90, 180),
                        deviation=15)

In [13]:
rg_angles.fit_transform(data)

,reg_length_AOI[aoi_2]_min,reg_length_AOI[aoi_2]_max,reg_speed_AOI[aoi_2]_mean,reg_speed_AOI[aoi_2]_kurtosis,reg_acceleration_AOI[aoi_2]_mean,reg_length_AOI[aoi_1]_min,reg_length_AOI[aoi_1]_max,reg_speed_AOI[aoi_1]_mean,reg_speed_AOI[aoi_1]_kurtosis,reg_acceleration_AOI[aoi_1]_mean
1_1,0.018059,0.304056,3.611862,0.003639,86.835422,NaN,NaN,NaN,NaN,NaN
1_2,0.015387,0.321837,4.565987,-1.584169,88.416342,0.030813,0.261686,2.160245,1.798463,29.761795
1_3,0.005249,0.265067,2.819047,0.600049,94.321225,0.254050,0.254050,6.466842,NaN,82.306756
1_4,0.009554,0.277172,3.581942,-0.401002,108.650739,0.272092,0.272092,4.515671,NaN,37.471339
1_5,0.021956,0.300800,3.058471,-0.957242,82.020561,0.084751,0.084751,0.667469,NaN,2.628390
...,...,...,...,...,...,...,...,...,...,...
2_33,0.007210,0.210085,3.374250,-0.225032,211.207507,0.080692,0.080692,6.478150,NaN,260.041351
2_34,0.021401,0.063878,3.868073,-1.623608,281.441170,NaN,NaN,NaN,NaN,NaN
2_35,0.007504,1.269198,5.681258,6.730013,321.060316,0.010291,0.203974,2.866314,NaN,143.354946
2_36,0.009415,0.078876,2.948550,2.758206,218.493133,0.017941,0.017941,2.553943,NaN,181.775335
